In [1]:

using POMDPs, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, Statistics
using FileIO, JLD2, TickTock

## State Functions

In [2]:
function state_cnt(n, S) #n = number of units; S = number of states
    if S==1
        return 1
    end
    return binomial(n+S-1, S-1)
end

function state_index(S,s) #s = state vector
    #S = length(s)
    if S == 1
        return 1
    end
    if s[S]==0
        return state_index(S-1, s[1:(S-1)])
    end
    
    n_prev = sum(s)
    prev = state_cnt(n_prev, S-1) # count of all states with s[S]=0
    inc = prev
    for i in 1:(s[S]-1)
        inc = inc/(n_prev+S-2)*n_prev #count of all states with s[S]=i
        prev = prev + inc
        n_prev = n_prev - 1
    end
    return prev + state_index(S-1, s[1:(S-1)])
end

function state_vec(n, S, ind) # n = number of units; S = number of states; ind = state index
    if ind < 0.5 || ind > state_cnt(n, S) + 0.5
        println("index is out of range!")
        return -1
    end
    if S==1 
        return [n]
    end
    if ind < state_cnt(n, S-1) + 0.5
        return push!(state_vec(n, S-1, ind), 0)
    end
    prev = state_cnt(n, S-1)
    inc = prev
    last_state = 0
    n_prev = n
    while ind > prev + 0.5
        inc = inc/(n_prev+S-2)*n_prev #count of all sta_tes with s[S]=last_state
        prev += inc
        n_prev -= 1
        last_state += 1
    end
    return push!(state_vec(n-last_state, S-1, ind - prev + inc), last_state)
end

#using hueristic rule for rollout
mutable struct nNRollout <: Policy
    n::Int64
    N::Int64
end

mutable struct nmNRollout <: Policy
    n::Int64
    m::Int64
    N::Int64
end

mutable struct mystate
    state::Vector{Int64};
end

In [3]:
function POMDPs.action(p::nNRollout, s::mystate)
        local_a = 0;
        local_s = s.state;
        nN = sample(1:Number_level, 2, replace = true);
        p.N = maximum(nN);
        p.n = minimum(nN);
        if sum(local_s[p.N:Number_level])>=1
        local_a = p.n;
        end
        return local_a; 
end

function POMDPs.action(p::nmNRollout, s::mystate)
    local_a = 0;
    local_s = s.state;
    nmN = sample(1:Number_level, 3, replace = true);
    p.N = maximum(nmN);
    p.n = minimum(nmN);
    p.m = sum(nmN)-p.N-p.n;
    if sum(local_s[p.N:Number_level])>=1 || sum(local_s[p.m:Number_level])>=2
        local_a = p.n;
    end
    return local_a; 
end

In [4]:
function findNn()
    println("Finding best nN policy...")

for N in 2:(Number_level)
    for n in 1:N
#         println("N = ",N, "  n = ",n);
        Threads.@threads  for j in 1:repetition
            history = sim(multiunit2,max_steps=simsteps,initialstate=mystate(state_vec(NumberUnits, Number_level, 1))) do s
            local_s = s.state;
            local_a = 0;
                if sum(local_s[N:Number_level])>=1 
                local_a = n;
                end
                return local_a; 
            end
            temp_rewards[j] = discounted_reward(history);
        end
        rewards_nN[n,N] = mean(temp_rewards);
        rewards_nN_std[n,N] = std(temp_rewards);
        
    end
end
(max_rward,nN) = findmax(rewards_nN);
println("Max rewards of (N, n) policy is ", max_rward, "  n is ", nN[1], "  N is ",nN[2]);
return  max_rward,rewards_nN_std[nN] ,nN;
end


findNn (generic function with 1 method)

In [5]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],mean=[],std=[],nN=[])

cost=[ [0	-200	-100	-1200 -2000],
    [0	-400	-100	-1200 -2000],
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 



for units in [60,50,40,30,25,20]
    for K in 2:2:convert(Int64, floor((units+1)/2))
        
    global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)
global Number_level = 10;
#action_level = 0;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
#fullname = "C:/Users/yongchen/OneDrive - University of Iowa/Codes/Julia/ReinforcementLearning/tm6.jld2"
#fullname = "C:/Users/yongchen/Dropbox/reinforcement learning for maintenance/Matlab/MultipleUnits/tm6.jld2";
global fullname = "tm10.jld2";
global Transition_matrix = load(fullname,"transition_matrix");



global state_number = state_cnt(NumberUnits,Number_level);
# global arrayofstates = Vector{mystate}(undef, state_number);

# Threads.@threads for i in 1:state_number
#     global arrayofstates[i]  = mystate(state_vec(NumberUnits, Number_level, i));
# end

global crd = Array{Categorical}(undef,Number_level)
for i in 1:Number_level
    global crd[i] = Categorical(Transition_matrix[i,:]);
end
    
    
    
global multiunit2 = QuickMDP(
    gen = function (s, a, rng)       #s is a vector of number units at each level and a is the number of units we will repair
        local_s = s.state;
        # println(local_s)
        # based on s, create a status vector corresponding to each unit
        degradation_state = repeat(1:1,NumberUnits);
        k = 1;
        for i in 1:Number_level
            for j in 1:local_s[i]
                degradation_state[k]=i;
                k = k+1;
            end
        end
        r = 0.0;
        number_reset=0
        prevent_repair = false;
        if a!=0
        number_reset = sum(local_s[a:Number_level]);
        end
        #using a for loop to compute next state for each unit
        if local_s[Number_level] >= limit && a==0
                        r+=system_penalty
                    number_reset=local_s[Number_level]
                    end 
        if local_s[Number_level] >= limit && a!=0
                        r+=system_penalty
                     number_reset = sum(local_s[a:Number_level]);
                    end 
        for i in 1:(NumberUnits-number_reset)  #a is the number of units we want to preventively repair
            #in this loop, all units continues
                    if degradation_state[i] != Number_level
            degradation_state[i] = rand(crd[degradation_state[i]]);
            r = r+normal_operation;
                    end
            # if degradation_state[i] == Number_level
            #     r+=system_penalty/limit
            # end
        end
        
        for i in (NumberUnits-number_reset+1):NumberUnits
                if degradation_state[i] == Number_level
                    r = r + failure_penalty  ;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                            end    
                    else
                    r = r + maintenance_penalty;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                end
                    end
            degradation_state[i] = rand(crd[1]); #reset status; add additional transition
            r = r+normal_operation; #add operation benefit
         end
        #collect degradation state to form the state
        sp = repeat(0:0,Number_level);
        for i in 1:NumberUnits
            sp[degradation_state[i]] = sp[degradation_state[i]]+1;
        end
        return (sp=mystate(sp), r=r)
    end,
    actions = 0:(Number_level-1), 
    actiontype = function()
        return Int64;
    end,
#     states = arrayofstates,
    initialstate = function()
        POMDPModelTools.ImplicitDistribution() do rng
            return (mystate(state_vec(NumberUnits, Number_level, 1)))
        end
    end, #all u #all units start fresh. Need to change according to unit number and level number. ##For simulation, we need to use ImplicitDistribution
    discount = 0.95,
    isterminal = false              # no ending
    )


    
    
    
for c in cost
        tick()
   println(units)  
             print("K : ")
                    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println(c)     
   global  simsteps = 100;
global repetition = 10000;
global rewards_nN = zeros(Number_level,Number_level);
global rewards_nN = rewards_nN.+(-100000000.0);
global rewards_nN_std = zeros(Number_level,Number_level);
global rewards_nN_std = rewards_nN_std.+(-100000000.0);

global discount_factor =0.95
global temp_rewards = zeros(repetition,1);
rewards,std, nN=findNn()
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,rewards,std, nN])
        tock()   
    end
 
    
        end   
    end

60
K : 2

[ Info:  started timer at: 2024-03-13T09:40:32.042



[0 -200 -100 -1200 -2000]
Finding best nN policy...
Max rewards of (N, n) policy is -50460.11309353139  n is 7  N is 7
60
K : 2
[0 -400 -100 -1200 -2000]
Finding best nN policy...


[ Info:          335.5007896s: 5 minutes, 35 seconds, 500 milliseconds
[ Info:  started timer at: 2024-03-13T09:46:07.690


Max rewards of (N, n) policy is -54242.808237289035  n is 7  N is 8
60
K : 2
[0 -600 -100 -1200 -2000]
Finding best nN policy...


...

[ Info:          154.8198343s: 2 minutes, 34 seconds, 819 milliseconds
[ Info:  started timer at: 2024-03-15T09:18:34.013



Finding best nN policy...
Max rewards of (N, n) policy is -44214.748514916195  n is 5  N is 10
20
K : 10
[0 -3200 -100 -1200 -2000]
Finding best nN policy...


[ Info:          154.2479893s: 2 minutes, 34 seconds, 247 milliseconds
[ Info:  started timer at: 2024-03-15T09:21:08.264


Max rewards of (N, n) policy is -47095.952517670026  n is 5  N is 10


[ Info:          154.7143205s: 2 minutes, 34 seconds, 714 milliseconds


In [6]:
df

Row,u,K,n,s,m,f,p,mean,std,nN
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20,10,0,-3200,-100,-1200,-2000,-47096.0,3413.02,"CartesianIndex(5, 10)"
2,20,10,0,-2800,-100,-1200,-2000,-44214.7,3170.08,"CartesianIndex(5, 10)"
3,20,10,0,-2400,-100,-1200,-2000,-41315.0,2922.69,"CartesianIndex(5, 10)"
4,20,10,0,-2000,-100,-1200,-2000,-38441.2,2725.02,"CartesianIndex(5, 10)"
5,20,10,0,-1800,-100,-1200,-2000,-36950.6,2623.11,"CartesianIndex(5, 10)"
6,20,10,0,-1600,-100,-1200,-2000,-35248.3,2479.3,"CartesianIndex(5, 9)"
7,20,10,0,-1400,-100,-1200,-2000,-33105.5,2364.91,"CartesianIndex(5, 9)"
8,20,10,0,-1200,-100,-1200,-2000,-31112.9,2287.95,"CartesianIndex(5, 9)"
9,20,10,0,-1000,-100,-1200,-2000,-29055.7,2160.89,"CartesianIndex(6, 9)"


In [7]:
using CSV
CSV.write("Homogeneous nN_simple.csv", df)

"Homogeneous nN_simple.csv"

In [8]:
#= using Plots
 =#

In [9]:
#= plot(df[df.s.==-250,:].u,df[df.s.==-250,:].mean) =#

In [10]:
#= convert(Int64,df.mean[1]) =#

In [11]:
#= plot(df.u,df.mean.+1100.0,yticks=df.mean,xticks=df.u,ytickfontrotation=10.0)
scatter!(df.u,df.mean.+1100.0)
plot!(xscale= :identity
    
    ,yscale=:log10,ytickfontrotation=10.0)
 =#

In [12]:
#= plot(df[df.s.==-300,:].u,df[df.s.==-300,:].mean,label="nN") =#